In [1]:
import pandas as pd
import numpy as np
import string
import scipy

import re
from pprint import pprint

import nltk
from nltk.stem import PorterStemmer

import gensim, logging
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel, ldamodel

import spacy
# For future ref, the only install that's ever worked for spacy: conda install -c conda-forge spacy

import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

In [2]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.tokenize import word_tokenize

In [3]:
duncan = open('duncan.txt','r')
#print(duncan.read())

duncan = [re.sub("\'", "", sent) for sent in duncan]
pprint(duncan[:1])

['Emily C. Duncan received two patents in 1903 and 1904 for banking-related '
 'calculators.[1] Duncan was born in 1849 in Coral, Illinois as Emily '
 'Cornelius Forbes and later resided in Centralia, Wisconsin (which later '
 'became Wisconsin Rapids) and Jennings, Louisiana. She invented new banking '
 'calculators and was mentioned in Inventive Age magazine in February 1905. '
 'Duncan was recognized as one of many women who worked in the age of '
 'technology before the actual computer.[2] She was married to inventor James '
 'Eugene Duncan and although he was believed to be the only inventor in the '
 'family written accounts from her granddaughter indicated she assisted her '
 'husband with numerous inventions.[3] Based on her written statement provided '
 'in the original patent application she stated that her goal was to provide a '
 '“simple and readily understood structure with which computations ordinarily '
 'requiring considerable time and care may be accurately and quickl

In [4]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))
 
data_words = list(sent_to_words(duncan))

print(data_words[:1])

[['emily', 'duncan', 'received', 'two', 'patents', 'in', 'and', 'for', 'banking', 'related', 'calculators', 'duncan', 'was', 'born', 'in', 'in', 'coral', 'illinois', 'as', 'emily', 'cornelius', 'forbes', 'and', 'later', 'resided', 'in', 'centralia', 'wisconsin', 'which', 'later', 'became', 'wisconsin', 'rapids', 'and', 'jennings', 'louisiana', 'she', 'invented', 'new', 'banking', 'calculators', 'and', 'was', 'mentioned', 'in', 'inventive', 'age', 'magazine', 'in', 'february', 'duncan', 'was', 'recognized', 'as', 'one', 'of', 'many', 'women', 'who', 'worked', 'in', 'the', 'age', 'of', 'technology', 'before', 'the', 'actual', 'computer', 'she', 'was', 'married', 'to', 'inventor', 'james', 'eugene', 'duncan', 'and', 'although', 'he', 'was', 'believed', 'to', 'be', 'the', 'only', 'inventor', 'in', 'the', 'family', 'written', 'accounts', 'from', 'her', 'granddaughter', 'indicated', 'she', 'assisted', 'her', 'husband', 'with', 'numerous', 'inventions', 'based', 'on', 'her', 'written', 'state

In [5]:
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)

/Users/marieskoczylas/anaconda3/envs/nlp_environ/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [6]:
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

print(trigram_mod[bigram_mod[data_words[0]]])

['emily', 'duncan', 'received', 'two', 'patents', 'in', 'and', 'for', 'banking', 'related', 'calculators', 'duncan', 'was', 'born', 'in', 'in', 'coral', 'illinois', 'as', 'emily', 'cornelius', 'forbes', 'and', 'later', 'resided', 'in', 'centralia', 'wisconsin', 'which', 'later', 'became', 'wisconsin', 'rapids', 'and', 'jennings', 'louisiana', 'she', 'invented', 'new', 'banking', 'calculators', 'and', 'was', 'mentioned', 'in', 'inventive', 'age', 'magazine', 'in', 'february', 'duncan', 'was', 'recognized', 'as', 'one', 'of', 'many', 'women', 'who', 'worked', 'in', 'the', 'age', 'of', 'technology', 'before', 'the', 'actual', 'computer', 'she', 'was', 'married', 'to', 'inventor', 'james', 'eugene', 'duncan', 'and', 'although', 'he', 'was', 'believed', 'to', 'be', 'the', 'only', 'inventor', 'in', 'the', 'family', 'written', 'accounts', 'from', 'her', 'granddaughter', 'indicated', 'she', 'assisted', 'her', 'husband', 'with', 'numerous', 'inventions', 'based', 'on', 'her', 'written', 'statem

In [7]:
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatizations(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://space.io/api/annotation"""
    
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
data_words_nostops = remove_stopwords(data_words)

data_words_bigrams = make_bigrams(data_words_nostops)

#nlp = spacy.load('en', disable=['parser', 'ner']) # Doesn't work because spacy and 'en' don't work

#data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_words_bigrams[:1])

[['emily', 'duncan', 'received', 'two', 'patents', 'banking', 'related', 'calculators', 'duncan', 'born', 'coral', 'illinois', 'emily', 'cornelius', 'forbes', 'later', 'resided', 'centralia', 'wisconsin', 'later', 'became', 'wisconsin', 'rapids', 'jennings', 'louisiana', 'invented', 'new', 'banking', 'calculators', 'mentioned', 'inventive', 'age', 'magazine', 'february', 'duncan', 'recognized', 'one', 'many', 'women', 'worked', 'age', 'technology', 'actual', 'computer', 'married', 'inventor', 'james', 'eugene', 'duncan', 'although', 'believed', 'inventor', 'family', 'written', 'accounts', 'granddaughter', 'indicated', 'assisted', 'husband', 'numerous', 'inventions', 'based', 'written', 'statement', 'provided', 'original', 'patent', 'application', 'stated', 'goal', 'provide', 'simple', 'readily', 'understood', 'structure', 'computations', 'ordinarily', 'requiring', 'considerable', 'time', 'care', 'may', 'accurately', 'quickly', 'made', 'objective', 'invention', 'provide', 'device', 'cha

In [9]:
id2word = corpora.Dictionary(data_words_bigrams)
texts = data_words_bigrams
corpus = [id2word.doc2bow(text) for text in texts]

print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 2), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 2), (13, 1), (14, 1), (15, 1), (16, 1), (17, 2), (18, 4), (19, 1), (20, 1), (21, 1), (22, 1), (23, 2), (24, 1), (25, 2), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 3), (33, 1), (34, 1), (35, 1), (36, 2), (37, 1), (38, 4), (39, 1), (40, 1), (41, 1), (42, 2), (43, 1), (44, 1), (45, 1), (46, 1), (47, 2), (48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 1), (56, 1), (57, 1), (58, 1), (59, 1), (60, 1), (61, 4), (62, 1), (63, 1), (64, 2), (65, 1), (66, 1), (67, 1), (68, 2), (69, 1), (70, 2), (71, 1), (72, 1), (73, 1), (74, 1), (75, 3), (76, 1), (77, 1), (78, 1), (79, 1), (80, 2), (81, 1), (82, 1), (83, 1), (84, 2), (85, 1), (86, 3), (87, 1), (88, 1), (89, 1), (90, 3), (91, 2), (92, 1), (93, 1), (94, 2), (95, 2), (96, 1), (97, 1), (98, 1), (99, 1), (100, 1), (101, 1), (102, 1), (103, 1), (104, 1), (105, 1), (106, 1), (107, 1), (108, 1), (109, 1), (110, 2)

In [10]:
[[(id2word[id], freq) for id, freq in cp] for cp in corpus[:1]]

[[('absolute', 1),
  ('accounts', 1),
  ('accuracy', 1),
  ('accurately', 1),
  ('actual', 1),
  ('age', 2),
  ('although', 1),
  ('another', 1),
  ('apparatus', 1),
  ('apparatuses', 1),
  ('application', 1),
  ('assisted', 1),
  ('banking', 2),
  ('based', 1),
  ('became', 1),
  ('believed', 1),
  ('born', 1),
  ('calculator', 2),
  ('calculators', 4),
  ('care', 1),
  ('cent', 1),
  ('centralia', 1),
  ('character', 1),
  ('computations', 2),
  ('computer', 1),
  ('computing', 2),
  ('considerable', 1),
  ('construction', 1),
  ('coral', 1),
  ('cornelius', 1),
  ('dates', 1),
  ('days', 1),
  ('designed', 3),
  ('detailed', 1),
  ('details', 1),
  ('determining', 1),
  ('device', 2),
  ('different', 1),
  ('duncan', 4),
  ('durable', 1),
  ('easily', 1),
  ('eight', 1),
  ('emily', 2),
  ('eugene', 1),
  ('explaining', 1),
  ('family', 1),
  ('february', 1),
  ('first', 2),
  ('forbes', 1),
  ('goal', 1),
  ('granddaughter', 1),
  ('hundreds', 1),
  ('husband', 1),
  ('illinois', 1

In [11]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20,
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.008*"operated" + 0.008*"original" + 0.008*"number" + 0.008*"numerous" + '
  '0.008*"objective" + 0.008*"one" + 0.008*"mentioned" + 0.008*"ordinarily" + '
  '0.008*"per" + 0.008*"patent"'),
 (1,
  '0.008*"duncan" + 0.008*"calculators" + 0.008*"invention" + 0.008*"age" + '
  '0.008*"two" + 0.008*"provide" + 0.008*"may" + 0.008*"later" + '
  '0.008*"designed" + 0.008*"emily"'),
 (2,
  '0.008*"operated" + 0.008*"original" + 0.008*"number" + 0.008*"numerous" + '
  '0.008*"objective" + 0.008*"one" + 0.008*"mentioned" + 0.008*"ordinarily" + '
  '0.008*"per" + 0.008*"patent"'),
 (3,
  '0.008*"operated" + 0.008*"original" + 0.008*"number" + 0.008*"numerous" + '
  '0.008*"objective" + 0.008*"one" + 0.008*"mentioned" + 0.008*"ordinarily" + '
  '0.008*"per" + 0.008*"patent"'),
 (4,
  '0.008*"operated" + 0.008*"original" + 0.008*"number" + 0.008*"numerous" + '
  '0.008*"objective" + 0.008*"one" + 0.008*"mentioned" + 0.008*"ordinarily" + '
  '0.008*"per" + 0.008*"patent"'),
 (5,
  '0.009*"

In [13]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
# A measure of how good the model is. The lower the better. Mine here is -6.44

coherence_model_lda = CoherenceModel(model=lda_model, texts=data_words_bigrams, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -6.44170537213736

Coherence Score:  0.57082816050018


In [14]:
pyLDAvis.enable_notebook() 
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word) 
vis

#print (pd.concat([df1, df2], sort=True))

/Users/marieskoczylas/anaconda3/envs/nlp_environ/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


TypeError: Object of type 'complex' is not JSON serializable

PreparedData(topic_coordinates=                                 x        y  topics  cluster       Freq
topic                                                                  
11      (-0.019758555608636322+0j)  (-0+0j)       1        1  99.695465
14     (-0.0004720848025382811+0j)       0j       2        1   0.019222
12     (-4.583917512499793e-06+0j)  (-0+0j)       3        1   0.019197
13      (0.0003080060622126998+0j)       0j       4        1   0.019164
10       (0.000488589584617799+0j)  (-0+0j)       5        1   0.019145
5       (0.0005248740858602829+0j)       0j       6        1   0.019138
6       (0.0008891729737140298+0j)       0j       7        1   0.019079
18      (0.0009354088943457972+0j)       0j       8        1   0.019062
19      (0.0009929990740180491+0j)       0j       9        1   0.019055
1       (0.0012068402829685514+0j)  (-0+0j)      10        1   0.018974
2       (0.0014889333370949884+0j)  (-0+0j)      11        1   0.013250
3       (0.0014889333370949884+0j